# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 4 2022 9:42AM,247819,10,129709,"Virtue Rx, LLC",Released
1,Oct 4 2022 9:39AM,247816,10,0086044087,ISDIN Corporation,Released
2,Oct 4 2022 9:39AM,247816,10,0086044652,ISDIN Corporation,Released
3,Oct 4 2022 9:39AM,247816,10,0086044832,ISDIN Corporation,Released
4,Oct 4 2022 9:39AM,247816,10,0086044929,ISDIN Corporation,Released
5,Oct 4 2022 9:39AM,247816,10,0086044963,ISDIN Corporation,Released
6,Oct 4 2022 9:39AM,247816,10,0086044965,ISDIN Corporation,Released
7,Oct 4 2022 9:39AM,247816,10,0086044973,ISDIN Corporation,Released
8,Oct 4 2022 9:39AM,247816,10,0086044974,ISDIN Corporation,Released
9,Oct 4 2022 9:39AM,247816,10,0086045018,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,247814,Executing,1
23,247816,Released,16
24,247817,Released,1
25,247818,Released,1
26,247819,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247814,NaN,1.0,NaN
247816,NaN,NaN,16.0
247817,NaN,NaN,1.0
247818,NaN,NaN,1.0
247819,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247719,0.0,0.0,1.0
247726,0.0,0.0,1.0
247736,0.0,0.0,2.0
247757,0.0,0.0,3.0
247764,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247719,0,0,1
247726,0,0,1
247736,0,0,2
247757,0,0,3
247764,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247719,0,0,1
1,247726,0,0,1
2,247736,0,0,2
3,247757,0,0,3
4,247764,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247719,,,1
1,247726,,,1
2,247736,,,2
3,247757,,,3
4,247764,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC"
1,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation
17,Oct 4 2022 9:39AM,247818,10,"Virtue Rx, LLC"
18,Oct 4 2022 9:38AM,247817,10,"Nextsource Biotechnology, LLC"
19,Oct 4 2022 9:27AM,247812,10,ISDIN Corporation
79,Oct 4 2022 9:26AM,247814,12,Hush Hush
80,Oct 4 2022 9:25AM,247813,12,Hush Hush
93,Oct 4 2022 9:12AM,247811,10,ISDIN Corporation
133,Oct 4 2022 8:55AM,247810,19,"Emersa Waterbox, LLC"
135,Oct 4 2022 8:53AM,247809,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC",,,1
1,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation,,,16
2,Oct 4 2022 9:39AM,247818,10,"Virtue Rx, LLC",,,1
3,Oct 4 2022 9:38AM,247817,10,"Nextsource Biotechnology, LLC",,,1
4,Oct 4 2022 9:27AM,247812,10,ISDIN Corporation,,,60
5,Oct 4 2022 9:26AM,247814,12,Hush Hush,,1,
6,Oct 4 2022 9:25AM,247813,12,Hush Hush,,8,5
7,Oct 4 2022 9:12AM,247811,10,ISDIN Corporation,,,40
8,Oct 4 2022 8:55AM,247810,19,"Emersa Waterbox, LLC",,,2
9,Oct 4 2022 8:53AM,247809,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC",1,,
1,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation,16,,
2,Oct 4 2022 9:39AM,247818,10,"Virtue Rx, LLC",1,,
3,Oct 4 2022 9:38AM,247817,10,"Nextsource Biotechnology, LLC",1,,
4,Oct 4 2022 9:27AM,247812,10,ISDIN Corporation,60,,
5,Oct 4 2022 9:26AM,247814,12,Hush Hush,,1,
6,Oct 4 2022 9:25AM,247813,12,Hush Hush,5,8,
7,Oct 4 2022 9:12AM,247811,10,ISDIN Corporation,40,,
8,Oct 4 2022 8:55AM,247810,19,"Emersa Waterbox, LLC",2,,
9,Oct 4 2022 8:53AM,247809,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC",1,,
1,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation,16,,
2,Oct 4 2022 9:39AM,247818,10,"Virtue Rx, LLC",1,,
3,Oct 4 2022 9:38AM,247817,10,"Nextsource Biotechnology, LLC",1,,
4,Oct 4 2022 9:27AM,247812,10,ISDIN Corporation,60,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC",1.0,NaN,NaN
1,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation,16.0,NaN,NaN
2,Oct 4 2022 9:39AM,247818,10,"Virtue Rx, LLC",1.0,NaN,NaN
3,Oct 4 2022 9:38AM,247817,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Oct 4 2022 9:27AM,247812,10,ISDIN Corporation,60.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC",1.0,0.0,0.0
1,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation,16.0,0.0,0.0
2,Oct 4 2022 9:39AM,247818,10,"Virtue Rx, LLC",1.0,0.0,0.0
3,Oct 4 2022 9:38AM,247817,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Oct 4 2022 9:27AM,247812,10,ISDIN Corporation,60.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4707961,189.0,1.0,4.0
12,495627,5.0,9.0,0.0
16,247796,8.0,0.0,0.0
19,495590,3.0,0.0,0.0
20,247719,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4707961,189.0,1.0,4.0
1,12,495627,5.0,9.0,0.0
2,16,247796,8.0,0.0,0.0
3,19,495590,3.0,0.0,0.0
4,20,247719,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,189.0,1.0,4.0
1,12,5.0,9.0,0.0
2,16,8.0,0.0,0.0
3,19,3.0,0.0,0.0
4,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,189.0
1,12,Released,5.0
2,16,Released,8.0
3,19,Released,3.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Completed,4.0,0.0,0.0,0.0,0.0
Executing,1.0,9.0,0.0,0.0,0.0
Released,189.0,5.0,8.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0
1,Executing,1.0,9.0,0.0,0.0,0.0
2,Released,189.0,5.0,8.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0
1,Executing,1.0,9.0,0.0,0.0,0.0
2,Released,189.0,5.0,8.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()